In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn.impute import SimpleImputer

from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_halving_search_cv  # Aktivieren
from sklearn.model_selection import HalvingRandomSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, GridSearchCV, TimeSeriesSplit, RandomizedSearchCV

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from google.colab import files
uploaded = files.upload() #hier muss man den merged_BSR_24Uhr hochladen

Saving Merged_BSR_24Uhr (1).csv to Merged_BSR_24Uhr (1).csv


In [5]:
import io

df = pd.read_csv(io.StringIO(uploaded['Merged_BSR_24Uhr (1).csv'].decode('utf-8')))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381661 entries, 0 to 381660
Data columns (total 91 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Schicht                                     381661 non-null  int64  
 1   Tour                                        381661 non-null  int64  
 2   Tonnage                                     381661 non-null  float64
 3   Temperature_Max (°C)                        381661 non-null  float64
 4   Rain_Sum (mm)                               381661 non-null  float64
 5   Snowfall_Sum (cm)                           381661 non-null  float64
 6   Wind_Speed_Max (km/h)                       381661 non-null  float64
 7   Daylight_Duration (s)                       381661 non-null  float64
 8   Temperature_Max (°C) 3-Day Avg              381661 non-null  float64
 9   Rain_Sum (mm) 3-Day Avg                     381661 non-null  float64
 

In [7]:
df.describe()

,Schicht,Tour,Tonnage,Temperature_Max (°C),Rain_Sum (mm),Snowfall_Sum (cm),Wind_Speed_Max (km/h),Daylight_Duration (s),Temperature_Max (°C) 3-Day Avg,Rain_Sum (mm) 3-Day Avg,...,Feiertag_Pfingstmontag_shift_1,Feiertag_Pfingstmontag_shift_2,Feiertag_Tag der Arbeit_shift_1,Feiertag_Tag der Arbeit_shift_2,Feiertag_Tag der Befreiung_shift_1,Feiertag_Tag der Befreiung_shift_2,Feiertag_Tag der Deutschen Einheit_shift_1,Feiertag_Tag der Deutschen Einheit_shift_2,Feiertag_Zweiter Weihnachtstag_shift_1,Feiertag_Zweiter Weihnachtstag_shift_2
count,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,...,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0
mean,1.018522,143.118574,12.125463,15.146716,1.643809,0.061519,19.949507,44466.500889,15.186362,1.519789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.134828,529.435676,5.993363,8.440446,3.558954,0.305219,6.834062,11275.498180,8.316128,2.168854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,1.000000,0.100000,-7.200000,0.000000,0.000000,7.900000,27516.100000,-6.633333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.000000,11.000000,8.080000,8.200000,0.000000,0.000000,15.100000,33653.080000,8.233333,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.000000,23.000000,11.100000,15.300000,0.200000,0.000000,18.800000,44861.230000,14.933333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.000000,41.000000,16.910000,21.600000,1.600000,0.000000,23.700000,55288.220000,22.000000,2.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2.000000,5513.000000,59.580000,37.700000,41.300000,4.900000,49.700000,60622.050000,33.900000,23.466667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df.drop(['   Brot', 'Fleisch', '    Fisch', '    Molkerei-Eier', '    Speisefette-öle', '    Obst', '    Gemüse',
         '    Zucker, Marmelade, Honig', '    Fertigerichte', '  Kaffee-Tee', '    Wasser-Saft'], axis=1, inplace=True)

In [9]:
X = df.drop(['Tonnage'], axis=1)
Y = df['Tonnage']

In [35]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381661 entries, 0 to 381660
Data columns (total 79 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Schicht                                     381661 non-null  int64  
 1   Tour                                        381661 non-null  int64  
 2   Temperature_Max (°C)                        381661 non-null  float64
 3   Rain_Sum (mm)                               381661 non-null  float64
 4   Snowfall_Sum (cm)                           381661 non-null  float64
 5   Wind_Speed_Max (km/h)                       381661 non-null  float64
 6   Daylight_Duration (s)                       381661 non-null  float64
 7   Temperature_Max (°C) 3-Day Avg              381661 non-null  float64
 8   Rain_Sum (mm) 3-Day Avg                     381661 non-null  float64
 9   Snowfall_Sum (cm) 3-Day Avg                 381661 non-null  float64
 

In [22]:
Y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 381661 entries, 0 to 381660
Series name: Tonnage
Non-Null Count   Dtype  
--------------   -----  
381661 non-null  float64
dtypes: float64(1)
memory usage: 2.9 MB


In [47]:
# TimeSeriesSplit für zeitabhängige Daten
tscv = TimeSeriesSplit(n_splits=4)

In [48]:
model = RandomForestRegressor()

In [51]:
param_dist = {
    'n_estimators': [12, 15, 18],  # Bessere Variation um den Wert 15
    'min_samples_split': [2, 4, 6],  # Etwas größere Werte, um zu prüfen, ob stabilere Splits den MSE weiter senken
    'min_samples_leaf': [1, 2, 3, 4],  # Variationen um 2 herum, um Overfitting zu vermeiden
    'max_features': ['sqrt', 'log2', None],  # Teste andere Werte für Features pro Split
    'max_depth': [None, 10, 15, 20],  # Mehr Tiefenvariationen testen
    'bootstrap': [True],
    'criterion': ['squared_error'],
    'warm_start': [True, False]  # Warm Start für das Wiederverwenden von Bäumen aus früheren Fit-Versuchen
}

In [52]:
search = HalvingRandomSearchCV(
    estimator=model,
    param_distributions=param_dist,
    factor=2,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=3
)

search.fit(X, Y)
print(search.best_estimator_)

n_iterations: 10
n_required_iterations: 10
n_possible_iterations: 16
min_resources_: 8
max_resources_: 381661
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 864
n_resources: 8
Fitting 4 folds for each of 864 candidates, totalling 3456 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 864 is smaller than n_iter=47707. Running 864 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


----------
iter: 1
n_candidates: 432
n_resources: 16
Fitting 4 folds for each of 432 candidates, totalling 1728 fits
----------
iter: 2
n_candidates: 216
n_resources: 32
Fitting 4 folds for each of 216 candidates, totalling 864 fits
----------
iter: 3
n_candidates: 108
n_resources: 64
Fitting 4 folds for each of 108 candidates, totalling 432 fits
----------
iter: 4
n_candidates: 54
n_resources: 128
Fitting 4 folds for each of 54 candidates, totalling 216 fits
----------
iter: 5
n_candidates: 27
n_resources: 256
Fitting 4 folds for each of 27 candidates, totalling 108 fits
----------
iter: 6
n_candidates: 14
n_resources: 512
Fitting 4 folds for each of 14 candidates, totalling 56 fits
----------
iter: 7
n_candidates: 7
n_resources: 1024
Fitting 4 folds for each of 7 candidates, totalling 28 fits
----------
iter: 8
n_candidates: 4
n_resources: 2048
Fitting 4 folds for each of 4 candidates, totalling 16 fits
----------
iter: 9
n_candidates: 2
n_resources: 4096
Fitting 4 folds for each of 

In [53]:
# Beste Parameter und Score ausgeben
print("Beste Parameter:", search.best_params_)
print("Bestes MSE:", -search.best_score_)

Beste Parameter: {'warm_start': True, 'n_estimators': 18, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 10, 'criterion': 'squared_error', 'bootstrap': True}
Bestes MSE: 15.682115429443275
